In [27]:
import cv2
import time
import numpy as np
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import HandTrackingModule as htm

In [28]:
wCam, hCam = 640, 480

In [29]:
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

In [30]:
detector = htm.handDetector()

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
volume.SetMasterVolumeLevel(0, None)
minVol = volRange[0]
maxVol = volRange[1]
vol = 0

In [31]:
while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        # print(lmList[4], lmList[8])
        
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1 + x2)//2, (y1 + y2)//2
        
        
        
        cv2.circle(img, (x1, y1), 10, (255, 0, 0), cv2.FILLED)
        cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
        cv2.circle(img, (cx, cy), 10, (255, 255, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
        
        length = math.hypot(x2 - x1, y2 - y1)
        # print(length)
        
        #hand range  195 - 25
        # volume range  -63.5 - 0
        
        vol = np.interp(length, [50, 300], [minVol, maxVol])
        print(int(length), vol)
        volume.SetMasterVolumeLevel(vol, None)
        
        if length < 25:
            cv2.circle(img, (cx, cy), 10, (0, 0, 0), cv2.FILLED)
    
    
    cv2.rectangle(img, (50, 150), (85, 400), (0, 255, 0), 3)
    cv2.rectangle(img, (50, int(vol)), (85, 400), (0, 255, 0), cv2.FILLED)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    
    cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    
    cv2.imshow('img', img)
    cv2.waitKey(1)

142 -39.974043063019025
123 -44.879626758290385
124 -44.54768627730351
118 -46.149534912084974
116 -46.716299689489446
102 -50.04909944189302
101 -50.29698087094865
97 -51.32225918617207
106 -49.05050092543142
102 -50.06760898807765
111 -47.904354398600475
112 -47.566639596442755
108 -48.57815690436281
115 -46.74804556570142
116 -46.66601415318277
116 -46.55046347748417
117 -46.438612868349026
121 -45.414613206912605
120 -45.5248331707878
119 -45.87065566155444
126 -44.05118478077303
127 -43.83418018958889
126 -44.094359810151744
129 -43.218122006168294
128 -43.48523385380846
130 -43.08245178157055
128 -43.48523385380846
124 -44.59664264670603
75 -57.116163383637975
63 -60.069500069743654
63 -59.95591061339541
52 -62.909220940817654
54 -62.45111349963205
33 -63.5
31 -63.5
28 -63.5
27 -63.5
25 -63.5
25 -63.5
22 -63.5
22 -63.5
20 -63.5
17 -63.5
19 -63.5
21 -63.5
21 -63.5
24 -63.5
24 -63.5
21 -63.5
32 -63.5
26 -63.5
19 -63.5
25 -63.5
26 -63.5
21 -63.5
21 -63.5
19 -63.5
30 -63.5
35 -63.5
2

KeyboardInterrupt: 